In [2]:
import sys
import sklearn
import tensorflow as tf
import numpy as np
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import keras as keras
from keras.layers import Conv2D, MaxPool2D,  \
    Dropout, Dense, Input, concatenate,      \
    GlobalAveragePooling2D, AveragePooling2D,\
    Flatten, BatchNormalization
from keras.initializers import glorot_uniform
from keras.initializers import Constant
from keras.utils import to_categorical
from keras.utils import np_utils
from keras.optimizers import SGD 
from keras.callbacks import LearningRateScheduler
import tensorflow as tf
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.datasets import cifar10, mnist, fashion_mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Rescaling, Resizing, RandomCrop, RandomFlip, RandomRotation
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
import cv2
from sklearn.model_selection import train_test_split
import math
from IPython.display import Image
import math 

2023-01-03 09:26:34.515634: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
import sys
import sklearn
import tensorflow as tf
import numpy as np
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.animation as animation

In [ ]:
env = gym.make('CartPole-v1', render_mode="rgb_array")
obs = env.reset()

In [3]:
n_inputs = 4
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(5, activation='elu', input_shape=[n_inputs]))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 5)                 25        
                                                                 
 dense_3 (Dense)             (None, 1)                 6         
                                                                 
Total params: 31
Trainable params: 31
Non-trainable params: 0
_________________________________________________________________


In [4]:
# one_step function
def one_step(env, obs, model, loss_fn) :
    with tf.GradientTape as tape :
        left_proba = model(np.array(obs)) # left_proba 0.8 
        action = (tf.random.uniform() > left_proba) #left_proba 크다 -> 오른쪽 80% 왼쪽 20%
        y_target = 1 - action # 왼쪽 80%, 오른쪽 20%
        loss = tf.reduce_mean(loss_fn(y_target, left_proba)) #MSE
    grads = tape.gradient(loss, model.trainable_variables)
    obs, reward, done, info = env.step(action)
    return obs, reward, done, grads
    

In [5]:
left_proba = model(obs[np.newaxis])
left_proba

TypeError: tuple indices must be integers or slices, not NoneType

In [ ]:
# episodes function, 여러 에피소드를 실행 했을 때 총 보상, 총 미분 값을 리턴해주는 함수 작성
# episodes function, 여러 에피소드를 실행했을 때 총 보상, 총 미분값을 리턴해주는 함수 작성
def multi_episodes(n_episodes, env, n_steps, model, loss_fn):
    all_rewards = []
    all_grads = []
    for episode in range(n_episodes):
        current_rewards = []
        current_grads = []
        obs = env.reset()
        for step in range(n_steps):
            obs, reward, done, grads = one_step(env, obs, model, loss_fn)
            current_rewards.append(reward)
            current_grads.append(grads)
            if done:
                break
        all_rewards.append(current_rewards)
        all_grads.append(current_grads)
    return all_rewards, all_grads
    

In [ ]:
# y = wx+b

# obs(x) -> left_probality(P(0, 1), 확률값)

# action -> 
#           True (1-P) = left
#           False (P)  = right

# y -> 1-action 
#                left = P
#               right = 1-P
# DL이 예측한 값이 y가 되는것.

# [flow]
# x -> P -> action -> step(action) -> y -> new obs -> x

# loss = "mse"

# 최종적으로 값이 고정되어 변하지 않는 상황이 발생. 반대의 값으로 출력.
# Reward * mse 
# 정규화를 통해 -, + 값을 입력 한뒤 방향을 출력.
# 랜덤한 값을 주어도 변하지 않을 때 까지 랜덤한 값을 준다.


### discount rate을 적용한 보상 함수 생성

In [14]:
def discount_rewards(rewards, discount_rate):
    discounted = np.array(rewards)
    for step in range(len(rewards) - 2, -1, -1):
        discounted[step] += discounted[step + 1] * discount_rate
    return discounted

In [25]:
discount_rewards([10, 0, -15], 0.8), discount_rewards([10, -15], 0.8)

(array([  0, -12, -15]), array([ -2, -15]))

### 보상을 정규화한 결과를 출력하는 함수 생성

In [19]:

def discount_and_normalize_rewards(all_rewards, discount_rate):
    all_discounted_rewards = [discount_rewards(rewards, discount_rate)
                              for rewards in all_rewards]
    flat_rewards = np.concatenate(all_discounted_rewards)
    reward_mean = flat_rewards.mean()
    reward_std = flat_rewards.std()
    return [(discounted_rewards - reward_mean) / reward_std
            for discounted_rewards in all_discounted_rewards]            
        

In [20]:
discount_and_normalize_rewards([[10, 0, -15], [10, 20]], 0.8)

[array([-0.22933216, -0.95353899, -1.13459069]),
 array([1.33978263, 0.97767921])]

In [ ]:
n_iterations = 150
n_episode = 10
n_steps = 200
